<h1 align='center'>Setting Grid Search for SVM</h1>
<h1 align='center'>Đỗ Trọng Khánh - 19521676</h1>

#Connect Driver

In [22]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Import Libary

In [43]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Prepare Data

In [44]:
data_link = '/gdrive/MyDrive/Colab Notebooks/Lập trình Python cho Máy học/Week 10/Social_Network_Ads.csv'
data = pd.read_csv(data_link)

In [45]:
data.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [46]:
df = pd.DataFrame(data)
X_data = df.iloc[:, 0:-1];
y_data = df.iloc[:, -1:];

In [47]:
X_data = X_data.to_numpy()
y_data = y_data.to_numpy()

# Split data and Scaler data

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaler = scaler.fit_transform(X_train)
X_val_scaler = scaler.fit_transform(X_val)
X_test_scaler = scaler.fit_transform(X_test)

# Grid Search for SVM

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Normal

In [31]:
best_C = 0
best_gamma = 0
best_acc = 0
best_kernel = ''

for c in [0.1, 1, 10, 100, 1000]:
    for g in [1, 0.1, 0.01, 0.001, 0.0001]:
        for k in ['linear', 'rbf', 'sigmoid']:
            model = SVC(C= c, gamma= g, kernel= k)
            model.fit(X_train_scaler, y_train)
            acc = model.score(X_val_scaler, y_val)
            if acc > best_acc:
                best_acc = acc
                best_C = c
                best_gamma = g
                best_kernel = k

In [32]:
print('\nBest C =', best_C, ', Best gamma = ', best_gamma, ', Best kernel = ', best_kernel)


Best C = 1 , Best gamma =  1 , Best kernel =  rbf


In [33]:
model = SVC(C = best_C, gamma = best_gamma, kernel = best_kernel)
model.fit(X_train_scaler, y_train)
y_pred = model.predict(X_test_scaler)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89        44
           1       0.91      0.81      0.85        36

    accuracy                           0.88        80
   macro avg       0.88      0.87      0.87        80
weighted avg       0.88      0.88      0.87        80



## KFlod

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2)

X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)

In [55]:
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle = True, random_state = 1)

In [56]:
best_C = 0
best_gamma = 0
best_acc = 0
best_kernel = ''

for c in [0.1, 1, 10, 100, 1000]:
    for g in [1, 0.1, 0.01, 0.001, 0.0001]:
        for k in ['linear', 'rbf', 'sigmoid']:
            accuracy_list = [];
            for train, val in kfold.split(X_train_scaler, y_train):
                X_train_kf, X_val_kf = X_train_scaler[train], X_train_scaler[val];
                y_train_kf, y_val_kf = y_train[train], y_train[val];

                model = SVC(C= c, gamma= g, kernel= k)
                model.fit(X_train_kf, y_train_kf)

                acc = model.score(X_val_kf, y_val_kf)
                accuracy_list.append(acc)
            acc_mean = np.mean(accuracy_list)
            if acc_mean > best_acc:
                best_acc = acc_mean
                best_C = c
                best_gamma = g
                best_kernel = k

In [57]:
print('\nBest C =', best_C, ', Best gamma = ', best_gamma, ', Best kernel = ', best_kernel)


Best C = 1 , Best gamma =  1 , Best kernel =  rbf


In [60]:
accuracy_list = []

for train, val in kfold.split(X_train_scaler, y_train):
    X_train_kf, X_val_kf = X_train_scaler[train], X_train_scaler[val]
    y_train_kf, y_val_kf = y_train[train], y_train[val]

    model = SVC(C = best_C, gamma = best_gamma, kernel = best_kernel)
    model.fit(X_train, y_train)

    acc = model.score(X_test, y_test)
    accuracy_list.append(acc)

print("Accuracy with KFold: ", np.mean(accuracy_list))

Accuracy with KFold:  0.9125
